In [1]:
from getTestsSeqPos import getListsOfProblemsSequential
gradedProblemsSeqArray, ungradedProblemsSeqArray, finalExamSeq=getListsOfProblemsSequential()

In [ ]:
from elasticsearch import Elasticsearch
import requests
from  elasticsearch.helpers import scan as scan

es = Elasticsearch([{"host": "localhost", "port": 9200}])

In [ ]:
import pandas as pd
#from pandas.io.json import json_normalize
#df=json_normalize(docs)

In [ ]:
body={
    "_source": [
        "context.user_id",
        "context.course_id",
        "referer",
        "time",
        "event.submission",
        "event.attempts",
        "event.problem_id" 
    ],
    "query": {
        "bool":{
            "must":[                
                {"match_phrase":{
                    "event_type": "problem_check"
                }},
                {"match":{
                    "event_source": "server"
                }}           
            ],
            "must_not":[
                {"match_phrase":{
                    "event_type": "problem_check_fail"
                }}
            ]            
        }
    }
}

In [ ]:
def ret():
    for hit in scan(es,
         index="online_courses", 
         query=body
        ):        
        if hit.get("_source").get("event") :
            if hit.get("_source").get("event").get("submission")!={}:
                submission=str(hit.get("_source").get("event").get("submission")).split('correct\': ')[1:]
                results=[]
                for res in submission:
                    if res[0:5]=='False':
                        results.append(0)
                    elif res[0:4]=='True':
                        results.append(1)
                yield {                    
                    "user_id":hit.get("_source").get("context").get("user_id"),
                    "course_id":hit.get("_source").get("context").get("course_id"),
                    "time":hit.get("_source").get("time"),
                    "results":results,
                    "attempt_num":hit.get("_source").get("event").get("attempts"),
                    "theme_id":hit.get("_source").get("referer").split('courseware/')[-1].split('/')[1],
                    "problem_id":hit.get("_source").get("event").get("problem_id"),
        }